#**Modified VGG16 Model Description**
***
This model is based on the **VGG16 architecture**, pre-trained on ImageNet, with modifications for binary classification.

### **Dataset Details**
- **Source:** BreaKHis dataset (40X, 100X, 200X, 400X magnification)
- **Classes:** Benign (0) and Malignant (1)
- **Split Ratio:**
  - **Training:** 60%
  - **Validation:** 10%
  - **Testing:** 30%

### **Model Architecture**
1. **Base Model:** VGG16 (pre-trained on ImageNet, with `include_top=False`)
2. **Frozen Layers:** All pre-trained layers are frozen to retain learned features.
3. **Custom Classifier:**
   - **Flatten Layer:** Converts feature maps to a 1D vector.
   - **Single Dense Layer (1 neuron, sigmoid activation):** Predicts the probability of malignancy.

### **Training Details**
- **Batch Size:** 16
- **Optimizer:** Adam
- **Loss Function:** Binary Cross-Entropy
- **Evaluation Metrics:** Accuracy, F1-Score, Geometric Mean Score (G-Mean), Informedness (IBA)
- **Epochs:** 10

### **Performance Evaluation**
- Model is trained on the training set and validated on the validation set.
- After training, the model is tested on the test set.
- Performance is measured using **accuracy, F1-score, G-Mean, and IBA**.

This model is designed for binary classification of histology slide images, leveraging the robust feature extraction capabilities of VGG16 with a minimal classification head to reduce overfitting.


***
**Magnification Factor: 40X**
***

In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/40X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/40X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total (1/7 of 70%)
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add modified classifier (Single Dense Layer)
x = Flatten()(base_model.output)
x = Dense(1, activation='sigmoid')(x)  # Single Dense Layer for Binary Classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 626
Total Malignant Images: 1370
Training samples: 1197
Validation samples: 200
Testing samples: 599
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 220s 7s/step - accuracy: 0.5759 - loss: 0.7906 - val_accuracy: 0.6850 - val_loss: 0.5747
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 888ms/step - accuracy: 0.7561 - loss: 0.5309 - val_accuracy: 0.7450 - val_loss: 0.4707
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 917ms/step - accuracy: 0.8033 - loss: 0.4386 - val_accuracy: 0.8300 - val_loss: 0.4279
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 892ms/step - accuracy: 0.8550 - loss: 0.3874 - val_accuracy: 0.8450 - val_loss: 0.4128
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 917ms/step - accuracy: 0.8910 - loss: 0.3500 - val_accuracy: 0.8550 - val_loss: 0.3946
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 892ms/step - accuracy: 0.9058 - loss: 0.3089 - val_accuracy: 0.8350 - val_loss: 0.3841
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/ste

***
**Magnification Factor: 100X**
***

In [3]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/100X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/100X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total (1/7 of 70%)
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add modified classifier (Single Dense Layer)
x = Flatten()(base_model.output)
x = Dense(1, activation='sigmoid')(x)  # Single Dense Layer for Binary Classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 649
Total Malignant Images: 1437
Training samples: 1251
Validation samples: 209
Testing samples: 626
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 283s 12s/step - accuracy: 0.6086 - loss: 0.7533 - val_accuracy: 0.8278 - val_loss: 0.5096
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.7409 - loss: 0.5188 - val_accuracy: 0.7943 - val_loss: 0.4395
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 38s 853ms/step - accuracy: 0.7973 - loss: 0.4261 - val_accuracy: 0.8469 - val_loss: 0.4002
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 845ms/step - accuracy: 0.8898 - loss: 0.3502 - val_accuracy: 0.8517 - val_loss: 0.3765
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 884ms/step - accuracy: 0.8976 - loss: 0.3182 - val_accuracy: 0.8469 - val_loss: 0.3601
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 882ms/step - accuracy: 0.9092 - loss: 0.2800 - val_accuracy: 0.8373 - val_loss: 0.3546
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 883ms/st

***
**Magnification Factor: 200X**
***

In [4]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/200X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/200X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total (1/7 of 70%)
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add modified classifier (Single Dense Layer)
x = Flatten()(base_model.output)
x = Dense(1, activation='sigmoid')(x)  # Single Dense Layer for Binary Classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 623
Total Malignant Images: 1390
Training samples: 1207
Validation samples: 202
Testing samples: 604
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 263s 9s/step - accuracy: 0.5819 - loss: 0.7912 - val_accuracy: 0.7030 - val_loss: 0.5336
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 800ms/step - accuracy: 0.7078 - loss: 0.5467 - val_accuracy: 0.7921 - val_loss: 0.4613
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 802ms/step - accuracy: 0.8377 - loss: 0.4115 - val_accuracy: 0.7822 - val_loss: 0.4346
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 812ms/step - accuracy: 0.8605 - loss: 0.3461 - val_accuracy: 0.7970 - val_loss: 0.4111
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 819ms/step - accuracy: 0.8862 - loss: 0.3142 - val_accuracy: 0.8366 - val_loss: 0.3952
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 820ms/step - accuracy: 0.9023 - loss: 0.2840 - val_accuracy: 0.8465 - val_loss: 0.3837
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 880ms/step - accuracy: 0.9178 - loss: 0.2639 - val_accura

***
**Magnification Factor: 400X**
***

In [5]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix

# Define dataset paths
benign_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/benign/400X"
malignant_dir = "/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/malignant/400X"

# Function to load image paths
def load_image_paths(dir_path):
    return [os.path.join(dir_path, img) for img in os.listdir(dir_path) if img.endswith('.png')]

benign_images = load_image_paths(benign_dir)
malignant_images = load_image_paths(malignant_dir)

print(f"Total Benign Images: {len(benign_images)}")
print(f"Total Malignant Images: {len(malignant_images)}")

# Create labels (0 = Benign, 1 = Malignant)
benign_labels = [0] * len(benign_images)
malignant_labels = [1] * len(malignant_images)

# Combine images and labels
all_images = np.array(benign_images + malignant_images)
all_labels = np.array(benign_labels + malignant_labels)

# Split into training (60%), validation (10%), and testing (30%)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.1429, stratify=train_labels, random_state=42  # 10% of total (1/7 of 70%)
)

print(f"Training samples: {len(train_images)}")
print(f"Validation samples: {len(val_images)}")
print(f"Testing samples: {len(test_images)}")

# Function to preprocess images
def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0
    return img, label

# Create TensorFlow datasets
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(process_path).shuffle(1000).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(process_path).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(process_path).batch(BATCH_SIZE)

# Ensure testing dataset is not empty
if sum(1 for _ in test_dataset) == 0:
    raise ValueError("Testing dataset is empty. Adjust your dataset split.")

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add modified classifier (Single Dense Layer)
x = Flatten()(base_model.output)
x = Dense(1, activation='sigmoid')(x)  # Single Dense Layer for Binary Classification

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
EPOCHS = 100
history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)

# Evaluate model
test_preds = model.predict(test_dataset)
test_preds = (test_preds > 0.5).astype(int).flatten()

# Get confusion matrix values
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()

# Calculate IBA
iba = (tp / (tp + fn)) + (tn / (tn + fp)) - 1

# Output results
f1 = f1_score(test_labels, test_preds)
gmean = geometric_mean_score(test_labels, test_preds)

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"G-Mean: {gmean:.4f}")
print(f"Informedness (IBA): {iba:.4f}")


Total Benign Images: 588
Total Malignant Images: 1232
Training samples: 1091
Validation samples: 183
Testing samples: 546
Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 168s 5s/step - accuracy: 0.5554 - loss: 0.8857 - val_accuracy: 0.6776 - val_loss: 0.8053
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 54s 820ms/step - accuracy: 0.7078 - loss: 0.6406 - val_accuracy: 0.6940 - val_loss: 0.5540
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 833ms/step - accuracy: 0.7185 - loss: 0.5486 - val_accuracy: 0.7432 - val_loss: 0.4810
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7642 - loss: 0.4534 - val_accuracy: 0.7923 - val_loss: 0.4372
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 38s 837ms/step - accuracy: 0.8247 - loss: 0.3789 - val_accuracy: 0.8415 - val_loss: 0.4208
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 828ms/step - accuracy: 0.8783 - loss: 0.3330 - val_accuracy: 0.8470 - val_loss: 0.4160
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 757ms/step - accuracy: 0.8961 - loss: 0.3195 - val_accuracy: 0.8579 - val_

4/5 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

5/5 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.8573 - loss: 0.3645
Test Accuracy: 0.8516
F1 Score: 0.8941
G-Mean: 0.8037
Informedness (IBA): 0.6232


# Here is a table summarizing the performance metrics for all magnification factors (40X, 100X, 200X, 400X) from notebook:

In [1]:
import pandas as pd

# Create a DataFrame with the provided values
data = {
    'Magnification': ['40X', '100X', '200X', '400X'],
    'Test Accuracy': [0.8681, 0.8626, 0.8742, 0.8516],
    'F1 Score': [0.9067, 0.9029, 0.9112, 0.8941],
    'G-Mean': [0.8221, 0.8163, 0.8308, 0.8037],
    'Informedness (IBA)': [0.6577, 0.6460, 0.6732, 0.6232]
}

df = pd.DataFrame(data)

# Display the table with formatting
styled_df = df.style \
    .format({
        'Test Accuracy': '{:.4f}',
        'F1 Score': '{:.4f}',
        'G-Mean': '{:.4f}',
        'Informedness (IBA)': '{:.4f}'
    }) \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#000000'), ('font-weight', 'bold'), ('color', 'white')]}
    ]) \
    .hide(axis='index')

styled_df


Magnification,Test Accuracy,F1 Score,G-Mean,Informedness (IBA)
40X,0.8681,0.9067,0.8221,0.6577
100X,0.8626,0.9029,0.8163,0.6460
200X,0.8742,0.9112,0.8308,0.6732
400X,0.8516,0.8941,0.8037,0.6232
